## 根画像抽出プログラム（太い部分の近似直線） 


基準方向の決定に上半分だけを用いることを除けば ver.1 と同じ

根部分のみ抽出、ただし、
- ヒゲ根は収縮膨張によりカットする。カット基準は、面積が2/3になるまで収縮し、その過程で消えた部分はカット。のち、同じ回数膨張させてほぼ元の根形に戻す。
- OpenCV の cv2.fitEllipse 関数で図形の近似楕円を求め、軸が垂直になるように回転する。

### 前提
- 概ね根が下になるように撮影された画像であることを前提としている。

### 結果
- シリンダ状、明確な楕円、三角状、逆三角状で線対称に近いものは良好な結果となる。

### わかっている問題点
- 球形は形状から軸を得るのは困難、また、角笛状のものはそもそも軸が曲線であり、楕円近似にそぐわない。
- 非常に稀に回転中心の位置が異常な値になることがある。その場合、描画領域外となって描画できず、
　輪郭線抽出ルーチンでエラーとなる。
 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import math
import pandas as pd
import os

# この関数は画像の入った複数のフォルダが１つのフォルダに入っていることを前提としている。
#  指定フォルダ直下の画像ファイルは無視する。直下のフォルダの中しか見ず、さらにサブフォルダがあっても再帰的に探すようなことはしない。

def listimage(path='シルエット', needThum=False):

    directory = os.listdir(path)
    
    # まずフォルダを全部リストアップ　→ folders
    folders = []
    for x in os.listdir(path):  
        if os.path.isdir(path + '/'+x) and x[0] != '.' and x[0] !='_':  #パスに取り出したオブジェクトを足してフルパスに
            folders.append(path + '/'+x)
    # print(folders)

    # folders の各フォルダの中にある jpg ファイルをリストアップ → ffiles
    # ffiles はフォルダごとのリストのリスト
    ffiles = []
    for x in folders:
        files = []
        for f in os.listdir(x):
            entry = x + '/'+f
            if os.path.isfile(entry) and f[-4:]=='.jpg' and f[0] != '.':
                files.append(entry)
        ffiles.append(files)
    
    if needThum: # サムネイルの作成
        i = 0
        for cat in ffiles:
            # print(folders[i])
            sam = Image.new('RGB', (800,600),(0,0,0))
            row = col = 0
            for rad in cat:
                img = Image.open(rad, 'r')
                thumbnail_size = (100, 100)
                img.thumbnail(thumbnail_size)
                sam.paste(img,(col,row))
                col += 100
                if col == 800:
                    col = 0
                    row += 100
                # plt.imshow(sam)
            sam.save('{}THUM.png'.format(folders[i]), 'PNG')
            print("saved {}".format(folders[i]))
            i +=1

    return folders,ffiles

# マージンをつける
def makemargin(img,mr=2):
    h,w = img.shape[:2]
    w2 = mr*w
    h2 = mr*h
    x1 = int((w2-w)/2)
    y1 = int((h2-h)/2)
    img2 = np.zeros((h2,w2),np.uint8)
    img2[y1:y1+h,x1:x1+w] = img
    return img2

# ２枚の画像をサイズを並べた画像を作成する
def mkparaimage(img1,img2):
    h1,w1 = img1.shape[:2]
    h2,w2 = img2.shape[:2]
    if img1.ndim == 2:
        img11 = np.zeros((h1,w1,3))
        img11[:,:,0]=img11[:,:,1]=img11[:,:,2]=img1
    else:
        img11=img1
    if img2.ndim == 2:
        img22 = np.zeros((h2,w2,3))
        img22[:,:,0]=img22[:,:,1]=img22[:,:,2]=img2
    else:
        img22=img2
    paraimg = 255*np.ones((max(h1,h2),w1+w2+10,3),dtype=np.uint8)
    
    paraimg[0:h1,0:w1,:] = img11
    paraimg[0:h2,w1+10:,:]=img22
    
    return paraimg

# mkparaimage で２枚並べた画像を表示
def imshowpara(img1,img2):
    plotimg(mkparaimage(img1,img2))

In [ ]:
# メインプログラム

def batch(images, savedir='概形シルエット3',interactive = True):
    # savedir 保存先
    # interactive 結果を１枚ずつ確認するかどうか
    
    for path in images:
        print(path)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img2 = makemargin(img) # 作業用のマージンを確保
    
        # ガウスぼかしを適用してシルエットを滑らかにする
        ksize = int(0.1*img.shape[1]/2)*2+1 # ぼかし量  元の図形の幅に応じて決める
        img2 = cv2.GaussianBlur(img2,(ksize,ksize),0) # ガウスぼかしを適用
        _ret,img2 = cv2.threshold(img2, 127, 255, cv2.THRESH_BINARY) # ２値化

        # 収縮・膨張によりヒゲ根を除去する
        area0 = np.sum(img2) # img2 の画素数*255 になるはず
        kernel = np.ones((3,3),np.uint8) 
        tmpimg = cv2.erode(img2,kernel,iterations = 1) # 収縮１回目
        area1 = np.sum(tmpimg) # 収縮したので area0 より少なくなる
        n = 1
        while 3*area1  > 2*area0: # 面積が 2/3 以下になるまで繰り返す
            tmpimg = cv2.erode(tmpimg,kernel,iterations = 1)
            area1 = np.sum(tmpimg) 
            n += 1
        # print("収縮・膨張回数",n)
        img3 = cv2.dilate(tmpimg,kernel,iterations = n) # 同じ回数膨張させる
        
        # 楕円近似で軸方向を求める
        ## あらためて輪郭を求め直す
        _img,cnt,hierarchy = cv2.findContours(img3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) #  あらためて輪郭を抽出
        
        
        

        ## 重心のy座標を求める
        ys = [y for [[x,y]] in cnt[0]]
        ymin = min(ys)
        ymax = max(ys)
        
        M=cv2.moments(cnt[0],True)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        
        # upperline = int(cy-3*(cy-ymin)/4)
        # lowerline = int(cy+3*(ymax-cy)/4)
        ## 輪郭線を重心より上と下に分割してそれぞれの重心のy座標を求める。
        upper = np.array([[[x,y] for [[x,y]] in cnt[0] if y < cy]])        
        lower = np.array([[[x,y] for [[x,y]] in cnt[0] if y > cy]])   
        MU = cv2.moments(upper,True)
        Uy = int(MU['m01']/MU['m00'])
        upperline = Uy-int((Uy - ymin)/2)
        Ux = int(MU['m10']/MU['m00'])
        ML = cv2.moments(lower,True)
        Ly = int(ML['m01']/ML['m00'])
        lowerline = Ly+int(0*(ymax - Ly)/4)
        Lx = int(ML['m10']/ML['m00'])
        
        ## 　重心の高さでの切断面の中点を求める。（本来は軸に対する垂直断面が望ましいが、軸がわかってない、
        ## 現在の状態での水平断面の中点で近似する）
        indexC = [i for i,[[x,y]] in enumerate(cnt[0]) if y == cy]
        pCx1 = cnt[0][indexC[0]][0][0]
        pCx2 = cnt[0][indexC[-1]][0][0]
        pCx = int((pCx1+pCx2)/2)
        
        # まず、基本調整量として、重心断面幅に対する重心と断面中心のズレに応じて lower, upper のラインを調整
        chouseiryo = int(0.2*np.abs((pCx-cx)/(pCx2-pCx1)*(Ly-Uy)))
        upperline += chouseiryo
        lowerline -= chouseiryo
        #  もしも、重心と 断面中点とのズレが大きい場合（Hornによくある） lowerline をズレと同程度重心の位置より上にあげる
        if np.abs(pCx-cx) > np.abs(pCx2-pCx1)/5:
            lowerline = cy - int(np.abs(pCx-cx))
            # 上半分の重心点の高さでの断面の中点を求める。
            indexU = [i for i,[[x,y]] in enumerate(cnt[0]) if y == Uy]
            pUx1 = cnt[0][indexU[0]][0][0]
            pUx2 = cnt[0][indexU[-1]][0][0]
            pUx = int((pUx1+pUx2)/2)
            # 断面幅に対する重心点と断面中点のズレに比例させて upperline と lowerline を近づける
            # (つまり、ズレの大きい部分を楕円近似の対象から外す
            chouseiryo = int(np.abs((pUx-Ux)/(pUx2-pUx1)*(cy-Uy)))
            upperline += chouseiryo
            lowerline -= chouseiryo
        else: 
            # 下半分の重心点の高さでの断面の中点を求める。
            indexL = [i for i,[[x,y]] in enumerate(cnt[0]) if y == Ly]
            pLx1 = cnt[0][indexL[0]][0][0]
            pLx2 = cnt[0][indexL[-1]][0][0]
            pLx = int((pLx1+pLx2)/2)
            # 断面幅に対する重心点と断面中点のズレに比例させて lowerline をあげる
            # (つまり、ズレの大きい部分を楕円近似の対象から外す
            lowerline -= int(np.abs((pLx-Lx)/(pLx2-pLx1)*(Ly - cy)))
            
            # 上半分の重心点の高さでの断面の中点を求める。
            indexU = [i for i,[[x,y]] in enumerate(cnt[0]) if y == Uy]
            pUx1 = cnt[0][indexU[0]][0][0]
            pUx2 = cnt[0][indexU[-1]][0][0]
            pUx = int((pUx1+pUx2)/2)
            # 断面幅に対する重心点と断面中点のズレに比例させて upperline を下げる
            # (つまり、ズレの大きい部分を楕円近似の対象から外す
            upperline += int(np.abs((pUx-Ux)/(pUx2-pUx1)*(cy-Uy)))

        testcon = np.array([[[x,y] for [[x,y]] in cnt[0] if y < lowerline and y > upperline]])

        # rect1 = cv2.minAreaRect(upper)  # 回転を考慮した矩形を求める場合
        # lowerline と upperline の間にある輪郭を対象に楕円近似する
        rect1 = cv2.fitEllipse(testcon) 
        box = np.int0(cv2.boxPoints(rect1))  # (対角点1,幅と高さ,回転角) から４点座標に変換
        # 回転角と中心
        deg = rect1[2]
        while deg > 45:
            deg -= 90 
        x0,y0 = rect1[0]
        #print("deg {0:0.1f} {1:0.1f} x0 {2:0.1f},y0 {3:0.1f}".format(rect1[2],deg,x0,y0))

        # 非常に稀であるが、回転すると全体が描画領域外に出ることがあるので作業領域を広く確保
        # mat = cv2.getRotationMatrix2D((x0,y0), deg, 1.0) # アフィン変換マトリクス
        img4 = makemargin(img3,mr=5) # 作業用のマージンを確保
        h3,w3 = img3.shape[:2]
        h4,w4 = img4.shape[:2]
        mat = cv2.getRotationMatrix2D((x0+(w4-w3)/2,y0+(h4-h3)/2), deg, 1.0) # アフィン変換マトリクス

        # アフィン変換の適用
        outimg = cv2.warpAffine(img4, mat, (0,0))
        # outimg = cv2.warpAffine(img3, mat, (0,0))

        # 再び最小矩形を求めて切り出す。ただし、マージンを５つける
        _nLabels, _labelImages, data, _center = cv2.connectedComponentsWithStats(outimg) 
        outimg = outimg[data[1][1]-5:data[1][1]+data[1][3]+5,data[1][0]-5:data[1][0]+data[1][2]+5]

        # 結果を保存する
        readdir,filename = os.path.split(path)
        _,subdir = os.path.split(readdir)
        os.makedirs(os.path.join(savedir,subdir), exist_ok=True) # 保存先フォルダがなければ作成
        cv2.imwrite(os.path.join(savedir,subdir,filename),outimg)
        
        if interactive:
            # upplerline と lowerline の断面中点を再度求める
            indexL = [i for i,[[x,y]] in enumerate(cnt[0]) if y == lowerline]
            pLx1 = cnt[0][indexL[0]][0][0]
            pLx2 = cnt[0][indexL[-1]][0][0]
            pLx = int((pLx1+pLx2)/2)
            indexU = [i for i,[[x,y]] in enumerate(cnt[0]) if y == upperline]
            pUx1 = cnt[0][indexU[0]][0][0]
            pUx2 = cnt[0][indexU[-1]][0][0]
            pUx = int((pUx1+pUx2)/2)            
            # 座標は作業用画像で求めているので、元の画像の原点に対応する座標を求める。
            pimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
            dx = int((img3.shape[1]-img.shape[1])/2)
            dy = int((img3.shape[0]-img.shape[0])/2)
            # 確認用情報の書き込み
            pimg = cv2.line(pimg,(0,upperline-dy),(img.shape[1],upperline-dy),(255,0,0),2)
            pimg = cv2.line(pimg,(0,lowerline-dy),(img.shape[1],lowerline-dy),(0,0,255),2) 
            pimg = cv2.line(pimg,(pUx-dx,upperline-dy),(pLx-dx,lowerline-dy),(255,255,0),2)   
            pimg = cv2.circle(pimg,(cx-dx,cy-dy),4,(255,0,0),2)
            pimg = cv2.circle(pimg,(Ux-dx,Uy-dy),4,(255,0,0),2)
            pimg = cv2.circle(pimg,(Lx-dx,Ly-dy),4,(255,0,0),2)
            cv2.imshow(filename,mkparaimage(pimg,outimg))
            key = cv2.waitKey(0)
            cv2.destroyAllWindows()
            cv2.waitKey(1) 
            if key == 113: #  "Q" で終了する
                break

In [ ]:
_dir,ffiles = listimage(path='シルエット', needThum=False) 
batch(sum(ffiles,[]), savedir='概形シルエット2',interactive = True)